### About **Spark**

Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes
(think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets
because each node only works with a small amount of data.

As each node works on its own subset of the total data, it also carries out a part of the total calculations required,
so that both data processing and computation are performed in parallel over the nodes in the cluster. It is a fact
that parallel computation can make certain types of programming tasks much faster.

When to use Spark? Answering questions like the following is helpful:

- Is my data too big to work with on a single machine?
- Can my calculations be easily parallelized?

Spark's core data structure is the Resilient Distributed Dataset (RDD). RDDs are hard to work with and one usually uses
Spark DataFrame abstraction built on top of RDDs. DataFrames are also more optimized for complicated operations than
RDDs.

#### Setup
To run spark locally on a windows machine, make sure to download `hadoop` [here](https://github.com/cdarlint/winutils)
and add the following to the environmental variables:
> `HADOOP_HOME=<your local hadoop folder (eg. C:\usr\bin\Hadoop\hadoop-3.2.2\bin)>


#### Session Initiation

In [5]:
from pyspark.sql import SparkSession
import pandas as pd

# create new spark session if necessary
spark = SparkSession.builder.\
    config("spark.driver.host", "127.0.0.1").\
    config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true").\
    config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true").\
    config("spark.sql.execution.arrow.pyspark.enabled", "true").\
    config("spark.sql.shuffle.partitions", "8").\
    config("spark.driver.memory", "8g").getOrCreate()
# list tables in cluster
spark.catalog.listTables()

root
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- area: double (nullable = true)
 |-- population: double (nullable = true)

+------------+------------+---------+-----+----------+
|country_code|     country|  capital| area|population|
+------------+------------+---------+-----+----------+
|          BR|      Brazil| Brasilia|8.516|     200.4|
|          RU|      Russia|   Moscow| 17.1|     143.5|
|          IN|       India|New Delhi|3.286|    1252.0|
|          CH|       China|  Beijing|9.597|    1357.0|
|          SA|South Africa| Pretoria|1.221|     52.98|
+------------+------------+---------+-----+----------+

+------------+------------+---------+-----+----------+
|country_code|     country|  capital| area|population|
+------------+------------+---------+-----+----------+
|          BR|      Brazil| Brasilia|8.516|     200.4|
|          RU|      Russia|   Moscow| 17.1|     143.5|
|          IN|       In

#### Reading / Importing Data

In [10]:
# 1. PySpark dataframe from pandas
# note: follow this process to make sure pyspark runs locally: https://github.com/cdarlint/winutils
dfp = pd.read_csv("data/brics.csv", index_col=0).rename_axis("country_code").reset_index()
dfs = spark.createDataFrame(dfp)
dfs.printSchema()
dfs.show()

# 2.
dfs_ = spark.read.csv("data/brics.csv", header=True)
dfs_ = dfs_.withColumnRenamed("_c0", "country_code")
dfs_.show()

root
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- capital: string (nullable = true)
 |-- area: double (nullable = true)
 |-- population: double (nullable = true)

+------------+------------+---------+-----+----------+
|country_code|     country|  capital| area|population|
+------------+------------+---------+-----+----------+
|          BR|      Brazil| Brasilia|8.516|     200.4|
|          RU|      Russia|   Moscow| 17.1|     143.5|
|          IN|       India|New Delhi|3.286|    1252.0|
|          CH|       China|  Beijing|9.597|    1357.0|
|          SA|South Africa| Pretoria|1.221|     52.98|
+------------+------------+---------+-----+----------+

+------------+------------+---------+-----+----------+
|country_code|     country|  capital| area|population|
+------------+------------+---------+-----+----------+
|          BR|      Brazil| Brasilia|8.516|     200.4|
|          RU|      Russia|   Moscow|17.10|     143.5|
|          IN|       In

#### Running SQL queries

In [7]:
# register the DF as a SQL temporary view
dfs.createOrReplaceTempView("population")
df_sql = spark.sql("SELECT * FROM population")
df_sql.show()

# convert results to pandas DF
df_sql.toPandas()



+------------+------------+---------+-----+----------+
|country_code|     country|  capital| area|population|
+------------+------------+---------+-----+----------+
|          BR|      Brazil| Brasilia|8.516|     200.4|
|          RU|      Russia|   Moscow| 17.1|     143.5|
|          IN|       India|New Delhi|3.286|    1252.0|
|          CH|       China|  Beijing|9.597|    1357.0|
|          SA|South Africa| Pretoria|1.221|     52.98|
+------------+------------+---------+-----+----------+



,country_code,country,capital,area,population
0,BR,Brazil,Brasilia,8.516,200.40
1,RU,Russia,Moscow,17.100,143.50
2,IN,India,New Delhi,3.286,1252.00
3,CH,China,Beijing,9.597,1357.00
4,SA,South Africa,Pretoria,1.221,52.98
